<a href="https://colab.research.google.com/github/kleeresearch/TextAnalysis/blob/master/Course/Day5/19_LR_sentiment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
import numpy as np

In [3]:
with open('/content/drive/MyDrive/KOSSDA_텍스트마이닝_강의자료/강의자료/Day4/python_code/Korean_movie_reviews_2016_small.txt', encoding='utf-8') as f:
    docs = [doc.strip().split('\t') for doc in f]
    docs = [(doc[0], int(doc[1])) for doc in docs if len(doc) == 2]
    texts, labels = zip(*docs) # 둘을 분리해서 별도의 list 변수로 저장

In [4]:
print(sum(labels))
print(len(labels))

11994
22909


In [5]:
from sklearn.model_selection import train_test_split
train_texts, test_texts, train_labels, test_labels = train_test_split(texts, labels, test_size=0.1, random_state=0)

In [6]:
# CounterVectorizer 클래스를 이용한 벡터 표현
from sklearn.feature_extraction.text import CountVectorizer
tf_vectorizer = CountVectorizer()
tf_train_features = tf_vectorizer.fit_transform(train_texts)
tf_test_features = tf_vectorizer.transform(test_texts)

TF IDF 값은 상대적으로 무엇을 더 많이 썼는지에 대한 절대적 빈도 정보

TF 정보를 사용하는 것이 더 적합한 젇보

각각에 대해서 모형 평가한 다음에 최종적으로 평가데이터에 대해서 모형 성능이 더 좋게 나오는 방법을 사용하면 됨.

일반적으로 TF에 대한 방법이 감성분석에서는 더 적합한 방법임.




In [7]:
from sklearn.linear_model import LogisticRegression

규제와 관련된 여러가지 파라미터들을 설정할 수 있음. 객체를 만들고 fit function을 이용해서 학습데이터를 학습. feature에 대한 학습, 종속변수에 대한 학습. 평가 데이터에 대해서 평가. predict function을 이용해서 각각의 영화평에 대한 sentiment를 예측. 모형의 성능을 파악하는데 accuracy를 계산. 분류문제의 경우에는 accuracy를 사용하지만 다른 문제도 동시에 리포트 해야 한다.



In [8]:
lr_tf_l1 = LogisticRegression(C=0.5, penalty='l1', solver='saga', max_iter=10000)

In [9]:
lr_tf_l1.fit(tf_train_features, train_labels) # 학습

LogisticRegression(C=0.5, max_iter=10000, penalty='l1', solver='saga')

In [10]:
pred_labels_tf_l1 = lr_tf_l1.predict(tf_test_features) # 예측

In [11]:
from sklearn.metrics import confusion_matrix

model evaluation metrics

- accuracy, recall, precision, F1 등 계산을 위해서 confusion matrix 를 사용

- 종속변수를 취하는 값이 positive 와 negative로 코딩.

컨퓨젼 매트릭스 해석

실제값이 행, 예측된 값이 열.
초록색으로된 2사분면값 top right 은 실제와 모형을 통해 예측한 값 = 그러한 관측치를 true negative 라고 한다. negative를 negative 라고 관측해서.

원래 종속변수의 값이 negative 인데 positive라고 예측해서 false positive.

원래 종속변수의 값이 positive인데 negative라고 모형이 예측해서 false negative.

원래 종변수값과 모형예측값도 positive를 positive라고 해서 true positive가 됨.

매트릭스는 관련된 관측치의 숫자가 된다.

모형이 제대로 관측한 수. 모형이 제대로 예측을 못한 관측치의 수. 이러한 행렬을 confusion matrix라고 한다. 이 매트릭스를 이용해서 분류모형의 지표인 accuracy, score값을 계산을 한다.

* accuracy는 전체 관측치 중에서 모형이 값ㄷ을 제대로 예측한 비중이다. 전체 관측치의 수는 전체 수를 다 더한 값을 분모 TN+TP+NN=NP, 초록색에 해당하는 제대로 관측한 수가 분자 TN + TP

* recall은 클래스별로 그 값이 존재한다. positive class 와 negative class가 있다. 원래의 종속변수 중에서 positive한 것중에서 positive라고 제대로 관측한 것. positive class 재현율은 positive라고 제대로 예측한 관측치의 비중. negative한 관측치 중에서 negative하다고 제대로 관측한 수. positive class에 대한 재현율이기 떄문에 원래의 종속변수의 값이 분모로 가야한다. 원래의 종속변수의 값이 positive한 것들, 분자에는 모형이 positive라고 제대로 예측한 관측치의 수가 분자로 가야 한다. positive class 에 대한 recall 값이다.

* 정밀도 precision은 positive class 와 negative class 값이 존재한다. 종속변수의 값을 positive 라고 예측한 값 중에서 positive 라고 관측한 비중이 정밀도가 됨. 우리가 선택한 모형이 종속변수의 값을 positive라고 예측한 관측치 중에서 실제 종속변수의 값이 positive인 값. negative라고 예측한 값 중에서 실제 negative 라고 예측한 값. 분자(TP)/분모(FP+TP)

* F1은 precision과 REC의 값.


- confusion matrix는 실제 종속변수의 값. true negative, true positive, false positive, false negative가 된다.


In [12]:
confusion_matrix(test_labels, pred_labels_tf_l1)

array([[ 871,  233],
       [ 126, 1061]])

In [13]:
from sklearn.metrics import classification_report

In [14]:
print(classification_report(test_labels, pred_labels_tf_l1))

              precision    recall  f1-score   support

           0       0.87      0.79      0.83      1104
           1       0.82      0.89      0.86      1187

    accuracy                           0.84      2291
   macro avg       0.85      0.84      0.84      2291
weighted avg       0.85      0.84      0.84      2291



In [15]:
# 하나의 영화평 예측하기
print(lr_tf_l1.predict_proba(tf_test_features[0]))
print(lr_tf_l1.predict(tf_test_features[0]))

[[0.30101145 0.69898855]]
[1]


In [16]:
# 단어 사전 만들기
words_dict = {}
for word, index in tf_vectorizer.vocabulary_.items():
    words_dict[index]=word

값이 큰 것을 기준으로 상위 30개, 부정을 기준으로 상위 30개 정리.

문서의 종속변수의 값이 1이 되는데 있어서 중요한 역할을 하는 것들. 소가로 안의 숫자들은 파라미터의 값을 의미. 파라미터 값이 클수록 긍정.

로지스틱 분석으로 감성분석뿐 아니라 어떠한 단어들이 중요한 역할을 하는지 순위 파악도 가능함.


만약 다른 분류 작업을 한다고 하면 코로나 19에 대해 정치 성향에 따라 분석한다고 하면...
계수값을 이용해서 종속병수의 값이 1(보수), 진보에서 많이 사용하는 단어들 찾을 수 있음.

https://www.dbpia.co.kr/pdf/pdfView.do?nodeId=NODE11057266&googleIPSandBox=false&mark=0&ipRange=false&b2cLoginYN=false&isPDFSizeAllowed=true&accessgl=Y&language=ko_KR&hasTopBanner=true


시대별 차이도 분석 가능.



In [17]:
# Get coefficients of the model
coefficients = lr_tf_l1.coef_.tolist()

sorted_coefficients = sorted(enumerate(coefficients[0]), key=lambda x:x[1], reverse=True)
# 학습에 사용된 각 단어마다의 coefficient (즉 weight) 값이 존재
# coefficient값이 큰 순으로 정렬 'reverse=True'


K=30
# print top K positive words
print("긍정 단어 상위 {} 개".format(str(K)))
for word_id, coef in sorted_coefficients[:K]:
    print('{0:} ({1:.3f})'.format(words_dict[word_id], coef))
# print top K negative words
print("\n부정 단어 상위 {} 개".format(str(K)))
for word_id, coef in sorted_coefficients[-K:]:
    print('{0:} ({1:.3f})'.format(words_dict[word_id], coef))

긍정 단어 상위 30 개
꿀잼 (3.447)
감탄 (2.836)
재밌게 (2.829)
재밌고 (2.561)
재미있게 (2.560)
존잼 (2.487)
재밌었 (2.404)
개꿀잼 (2.367)
여운 (2.349)
지루하지 (2.314)
재밌어 (2.230)
감사합 (2.202)
재미있었 (2.163)
빠르 (2.158)
흥미진진 (2.062)
강추 (2.055)
최고 (1.950)
사랑해 (1.947)
재미있어 (1.900)
재밋어 (1.864)
즐겁 (1.841)
유쾌 (1.841)
아름다운 (1.826)
재미있네 (1.818)
심장 (1.800)
낮아 (1.788)
감명 (1.755)
재미있고 (1.749)
재미있 (1.746)
재밌는 (1.704)

부정 단어 상위 30 개
갑자기 (-1.960)
지루해서 (-1.978)
댓글알바 (-1.991)
알바 (-2.028)
그닥 (-2.065)
지루하다 (-2.072)
별로 (-2.098)
왜곡 (-2.142)
졸았 (-2.143)
지루하고 (-2.147)
전형 (-2.161)
허무 (-2.184)
억지 (-2.200)
아까 (-2.234)
신파극 (-2.257)
짜증 (-2.308)
아까운 (-2.352)
재미없 (-2.366)
엉망 (-2.405)
망작 (-2.499)
지루했 (-2.501)
팔이 (-2.522)
역사왜곡 (-2.674)
거르 (-2.675)
쓰레기 (-2.737)
아까웠 (-2.802)
이하 (-3.005)
차라리 (-3.336)
노잼 (-3.620)
최악 (-4.687)


In [18]:
lr_tf_l2 = LogisticRegression(C=0.5, penalty='l2', solver='saga', max_iter=10000)

In [19]:
lr_tf_l2.fit(tf_train_features, train_labels) # 학습

LogisticRegression(C=0.5, max_iter=10000, solver='saga')

In [20]:
pred_labels_tf_l2 = lr_tf_l2.predict(tf_test_features)

In [21]:
print(classification_report(test_labels, pred_labels_tf_l2))

              precision    recall  f1-score   support

           0       0.87      0.83      0.85      1104
           1       0.85      0.88      0.86      1187

    accuracy                           0.86      2291
   macro avg       0.86      0.86      0.86      2291
weighted avg       0.86      0.86      0.86      2291



macro 평균

weighted 평균(가중평균): 젼채 관측치 중에서 평균을 따로 구한 것.

predict 기능을 사용하면 0과 1 중에서 예측 가능. 종속변수가 0을 취할 확률, 1을 취할 확률에 대해서 정보를 확인하기 위해 predict_probability 약자인 proba 라는 function을 사용하면 됨. 0.3은 종속변수가 0일 확률,

뭐가 더 성능이 좋은지 판단하는 기준은 선행연구를 기준으로 판단 가능

In [22]:
# Get coefficients of the model
coefficients = lr_tf_l2.coef_.tolist()

sorted_coefficients = sorted(enumerate(coefficients[0]), key=lambda x:x[1], reverse=True)
# 학습에 사용된 각 단어마다의 coefficient (즉 weight) 값이 존재
# coefficient값이 큰 순으로 정렬 'reverse=True'


K=30
# print top K positive words
print("긍정 단어 상위 {} 개".format(str(K)))
for word_id, coef in sorted_coefficients[:K]:
    print('{0:} ({1:.3f})'.format(words_dict[word_id], coef))
# print top K negative words
print("\n부정 단어 상위 {} 개".format(str(K)))
for word_id, coef in sorted_coefficients[-K:]:
    print('{0:} ({1:.3f})'.format(words_dict[word_id], coef))

긍정 단어 상위 30 개
재밌게 (2.619)
꿀잼 (2.497)
재미있게 (2.241)
재밌었 (2.197)
재밌어 (2.055)
여운 (1.967)
재미있었 (1.916)
최고 (1.906)
재밌고 (1.835)
강추 (1.782)
감사합 (1.721)
지루하지 (1.704)
감탄 (1.674)
재미있어 (1.655)
존잼 (1.585)
아름다운 (1.576)
재밌는 (1.564)
개꿀잼 (1.529)
재미있 (1.522)
가는 (1.511)
흥미진진 (1.509)
재밋어 (1.501)
재밌 (1.476)
가슴 (1.466)
지루할 (1.409)
재미있네 (1.403)
즐겁 (1.375)
대박 (1.370)
유쾌 (1.357)
심장 (1.331)

부정 단어 상위 30 개
비추 (-1.439)
지루하다 (-1.521)
지루해서 (-1.541)
아깝 (-1.583)
지루하고 (-1.617)
유치 (-1.618)
그닥 (-1.641)
엉망 (-1.645)
졸았 (-1.654)
자고 (-1.663)
실망 (-1.690)
허무 (-1.705)
아까웠 (-1.719)
거르 (-1.723)
아까 (-1.744)
팔이 (-1.770)
망작 (-1.785)
역사왜곡 (-1.817)
알바 (-1.833)
짜증 (-1.884)
아까운 (-1.909)
별로 (-2.026)
지루했 (-2.028)
억지 (-2.028)
이하 (-2.050)
재미없 (-2.271)
차라리 (-2.290)
쓰레기 (-2.395)
노잼 (-3.173)
최악 (-3.653)


로지스틱 회귀분석으로 학습을 하게 되면 파라미터로 최적값을 찾게 됨. 고유한 파라미터값을 가진다. 파라미터의 값이 bi가 음수이면 해당 단어와 종속변수의 값이 음수면 부정의 관계, 양수면 긍정의 관계가 있다. 파라미터의 값을 확인하게 되면 어떠한 단어가 부정이고 긍정인지 파악할 수 있을 뿐만 아니라 상대적으로 더 많은 부정적인 역할을 하는지 파악할 수 있음.

네트워크 분석을 하면 특정한 형용사/부사 등을 파악할 수 있어 유용하게 사용 가능.
